

# Homework 2 - Recurrent Neural Networks

In this part of the homework we are going to work with Recurrent Neural Networks, in particular GRU. One of the greatest things that Recurrent Neural Networks can do when working with sequences is retaining data from several timesteps in the past. We are going to explore that property by constructing an 'echo' Recurrent Neural Network.

The goal here is to make a model that given a sequence of letters or digits will output that same sequence, but with a certain delay. Let's say the input is a string 'abacaba', we want the model to not output anything for 3 steps (delay length), and then output the original string step by step, except the last 3 characters. So, target output is then 'XXXabac', where 'X' is empty output.

This is similar to [this notebook](https://github.com/Atcold/pytorch-Deep-Learning/blob/master/09-echo_data.ipynb) (which you should refer to when doing this assignment), except we're working not with a binary string, but with a sequence of integers between 0 and some N. In our case N is 26, which is the number of letters in the alphabet.

## Dataset

Let's implement the dataset. In our case, the data is basically infinite, as we can always generate more examples on the fly, so there's no need to load it from disk.

In [1]:
import random
import string

import torch

# Max value of the generated integer. 26 is chosen becuase it's
# the number of letters in English alphabet.
N = 26


def idx_to_onehot(x, k=N+1):
  """ Converts the generated integers to one-hot vectors """
  ones = torch.sparse.torch.eye(k)
  shape = x.shape
  res = ones.index_select(0, x.view(-1).type(torch.int64))
  return res.view(*shape, res.shape[-1])


class EchoDataset(torch.utils.data.IterableDataset):

  def __init__(self, delay=4, seq_length=15, size=1000):
    self.delay = delay
    self.seq_length = seq_length
    self.size = size

  def __len__(self):
    return self.size

  def __iter__(self):
    """ Iterable dataset doesn't have to implement __getitem__.
        Instead, we only need to implement __iter__ to return
        an iterator (or generator).
    """
    for _ in range(self.size):
      seq = torch.tensor([random.choice(range(1, N + 1)) for i in range(self.seq_length)], dtype=torch.int64)
      result = torch.cat((torch.zeros(self.delay), seq[:self.seq_length - self.delay])).type(torch.int64)
      yield seq, result

DELAY = 4
DATASET_SIZE = 200000
ds = EchoDataset(delay=DELAY, size=DATASET_SIZE)

## Model

Now, we want to implement the model. For our purposes, we want to use GRU. The architecture consists of GRU and a decoder. Decoder is responsible for decoding the GRU hidden state to yield a predicting for the next output. The parts you are responsible for filling with your code are marked with `TODO`.

In [2]:
class GRUMemory(torch.nn.Module):

  def __init__(self, hidden_size):
    super().__init__()
    #TODO: initialize your submodules
    self.gru = torch.nn.GRU(input_size=N + 1, hidden_size=hidden_size, batch_first=True)
    self.decoder = torch.nn.Linear(hidden_size, N + 1)

  def forward(self, x):
    # inputs: x - input tensor of shape (batch_size, seq_length, N+1)
    # returns:
    # logits (scores for softmax) of shape (batch size, seq_length, N + 1)
    # TODO implement forward pass
    h, _ = self.gru(x)                 # (batch_size, seq_length, hidden_size)
    logits = self.decoder(h)           # (batch_size, seq_length, N + 1)
    return logits

  @torch.no_grad()
  def test_run(self, s):
    # This function accepts one string s containing lowercase characters a-z.
    # You need to map those characters to one-hot encodings,
    # then get the result from your network, and then convert the output
    # back to a string of the same length, with 0 mapped to ' ',
    # and 1-26 mapped to a-z.

    # TODO
    # map input chars to indices (a->1, ..., z->26; anything else -> 0)
    idxs = torch.tensor(
        [ord(c) - 96 if ('a' <= c <= 'z') else 0 for c in s],
        dtype=torch.int64
    )  # (seq_length,)

    # one-hot encode and add batch dimension
    x = idx_to_onehot(idxs)            # (seq_length, N + 1)
    x = x.unsqueeze(0).to(next(self.parameters()).device)  # (1, seq_length, N + 1)

    # forward pass
    logits = self.forward(x)           # (1, seq_length, N + 1)
    pred_idx = logits.argmax(dim=-1).squeeze(0).tolist()  # (seq_length,)

    # map indices back to string (0 -> space, 1-26 -> a-z)
    out_chars = [(' ' if i == 0 else chr(i + 96)) for i in pred_idx]
    return ''.join(out_chars)

## Training
Below you need to implement the training of the model. We give you more freedom as for the implementation. The two limitations are that it has to execute within 10 minutes, and that error rate should be below 1%.

In [3]:
def test_model(model, sequence_length=15):
  """
  This is the test function that runs 100 different strings through your model,
  and checks the error rate.
  """
  total = 0
  correct = 0
  for i in range(500):
    s = ''.join([random.choice(string.ascii_lowercase) for i in range(random.randint(15, 25))])
    result = model.test_run(s)
    for c1, c2 in zip(s[:-D], result[D:]):
      correct += int(c1 == c2)
    total += len(s) - D

  return correct / total

In [4]:

import torch
from torch import nn
from torch.utils.data import DataLoader

import time
start_time = time.time()

# TODO

# keep delay consistent with the dataset / test
D = DELAY

def one_epoch(model, loader, optimizer, device):
    model.train()
    total_loss = 0.0
    criterion = nn.CrossEntropyLoss()
    for seq, tgt in loader:
        # 1) Keep seq on CPU for one-hot
        x = idx_to_onehot(seq, k=N+1).float()   # CPU

        # 2) Move tensors to target device AFTER one-hot
        x   = x.to(device)                      # (B, L, N+1)
        tgt = tgt.to(device).long()             # (B, L)

        logits = model(x)                       # (B, L, N+1)
        loss = criterion(logits.reshape(-1, N+1), tgt.view(-1))

        optimizer.zero_grad(set_to_none=True)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
        optimizer.step()

        total_loss += loss.item()
    return total_loss / max(1, len(loader))


device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
torch.backends.cudnn.benchmark = True
torch.manual_seed(0)

hidden_size = 128
model = GRUMemory(hidden_size).to(device)

# a reasonably fast and stable setup
batch_size = 256
num_workers = 0  # IterableDataset: keep 0 for simplicity / determinism
loader = DataLoader(ds, batch_size=batch_size, num_workers=num_workers)

optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

# Train for a small number of epochs; echo is easy to learn
# This should converge well under 10 minutes on CPU or GPU.
epochs = 2
for ep in range(epochs):
  loss = one_epoch(model, loader, optimizer, device)

end_time = time.time()
duration = end_time - start_time
accuracy = test_model(model)

print('duration:', duration)
print('accuracy:', accuracy)

assert duration < 600, 'execution took f{duration:.2f} seconds, which longer than 10 mins'
assert accuracy > 0.99, f'accuracy is too low, got {accuracy}, need 0.99'
print('tests passed')

duration: 21.5095534324646
accuracy: 1.0
tests passed


## Variable delay model

Now, to make this more complicated, we want to have varialbe delay. So, now, the goal is to transform a sequence of pairs (character, delay) into a character sequence with given delay. Delay is constant within one sequence.

### Dataset
As before, we first implement the dataset:

In [5]:
class VariableDelayEchoDataset(torch.utils.data.IterableDataset):

  def __init__(self, max_delay=8, seq_length=20, size=1000):
    self.max_delay = max_delay
    self.seq_length = seq_length
    self.size = size

  def __len__(self):
    return self.size

  def __iter__(self):
    for _ in range(self.size):
      seq = torch.tensor([random.choice(range(1, N + 1)) for i in range(self.seq_length)], dtype=torch.int64)
      delay = random.randint(0, self.max_delay)
      result = torch.cat((torch.zeros(delay), seq[:self.seq_length - delay])).type(torch.int64)
      yield seq, delay, result

### Model

And the model.

In [6]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from typing import List

class VariableDelayGRUMemory(nn.Module):
    def __init__(self, hidden_size, max_delay):
        super().__init__()
        self.hidden_size = hidden_size
        self.max_delay = int(max_delay)

        # Assumes global N is defined elsewhere.
        self.num_classes = (N + 1)

        delay_embed_dim = 8
        num_layers = 2

        self.delay_emb = nn.Embedding(self.max_delay + 1, delay_embed_dim)
        in_size = self.num_classes + delay_embed_dim

        self.gru = nn.GRU(
            input_size=in_size,
            hidden_size=hidden_size,
            num_layers=num_layers,
            batch_first=True,
        )
        self.decoder = nn.Linear(hidden_size, self.num_classes)

    def forward(self, x, delays):
        """
        x: (B, T, N+1) one-hot (float or long convertible)
        delays: (B,) integer delays in [0, max_delay]
        returns: (B, T, N+1) logits
        """
        # Normalize dtype
        if x.dtype not in (torch.float32, torch.float64):
            x = x.float()

        # Ensure delays are valid and long
        delays = delays.long().clamp_(0, self.max_delay)  # in-place is fine

        # Delay features: (B, De) -> (B, 1, De) -> (B, T, De)
        delay_emb = self.delay_emb(delays)                                   # (B, De)
        delay_feat = delay_emb.unsqueeze(1).expand(-1, x.size(1), -1)        # (B, T, De)

        # RNN forward
        gru_input = torch.cat([x, delay_feat], dim=-1)                       # (B, T, N+1+De)
        h_seq, _ = self.gru(gru_input)                                       # (B, T, H)

        return self.decoder(h_seq)                                           # (B, T, N+1)

    @torch.no_grad()
    def test_run(self, s, delay):
        """
        s: string of 'a'..'z' (others become blank token 0)
        delay: int
        returns: decoded string of predicted tokens (' ' for 0)
        """
        self.eval()
        device = next(self.parameters()).device

        # Map characters to indices: 0 = blank, 1..26 = 'a'..'z'
        idxs: List[int] = [
            (ord(ch) - ord('a') + 1) if ('a' <= ch <= 'z') else 0
            for ch in s
        ]

        x_idx = torch.tensor(idxs, dtype=torch.long, device=device).unsqueeze(0)  # (1, T)
        x_oh = F.one_hot(x_idx, num_classes=self.num_classes).float()             # (1, T, N+1)

        d = torch.tensor([int(delay)], dtype=torch.long, device=device)           # (1,)
        logits = self.forward(x_oh, d)                                            # (1, T, N+1)

        pred = logits.argmax(dim=-1)[0].tolist()
        return ''.join((' ' if t == 0 else chr(ord('a') + t - 1)) for t in pred)

### Train

As before, you're free to do what you want, as long as training finishes within 10 minutes and accuracy is above 0.99 for delays between 0 and 8.

In [7]:
def test_variable_delay_model(model, seq_length=20):
  """
  This is the test function that runs 100 different strings through your model,
  and checks the error rate.
  """
  total = 0
  correct = 0
  for i in range(500):
    s = ''.join([random.choice(string.ascii_lowercase) for i in range(seq_length)])
    d = random.randint(0, model.max_delay)
    result = model.test_run(s, d)
    if d > 0:
      z = zip(s[:-d], result[d:])
    else:
      z = zip(s, result)
    for c1, c2 in z:
      correct += int(c1 == c2)
    total += len(s) - d

  return correct / total

In [8]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
import time

start_time = time.time()

MAX_DELAY = 8
SEQUENCE_LEN = 20

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Data
train_ds = VariableDelayEchoDataset(
    max_delay=MAX_DELAY,
    seq_length=SEQUENCE_LEN,
    size=100_000
)
train_loader = DataLoader(
    train_ds,
    batch_size=256,
    num_workers=2 if device.type == "cuda" else 0,
    pin_memory=(device.type == "cuda"),
)

# Model & training utils
hidden_size = 256
model = VariableDelayGRUMemory(hidden_size=hidden_size, max_delay=MAX_DELAY).to(device)

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.AdamW(model.parameters(), lr=2e-3, weight_decay=1e-4)

EPOCHS = 6
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=EPOCHS)
clip_val = 1.0

scaler = torch.cuda.amp.GradScaler(enabled=(device.type == "cuda"))
num_classes = N + 1

def train_one_epoch() -> float:
    model.train()
    total_loss, batches = 0.0, 0
    for x_int, d_int, y_int in train_loader:
        x_int = x_int.to(device, non_blocking=True)
        d_int = d_int.to(device, non_blocking=True)
        y_int = y_int.to(device, non_blocking=True)

        x_oh = F.one_hot(x_int, num_classes=num_classes).float()  # (B,T,C)

        optimizer.zero_grad(set_to_none=True)
        with torch.cuda.amp.autocast(enabled=(device.type == "cuda")):
            logits = model(x_oh, d_int)
            loss = criterion(logits.reshape(-1, num_classes), y_int.reshape(-1))

        scaler.scale(loss).backward()
        nn.utils.clip_grad_norm_(model.parameters(), clip_val)
        scaler.step(optimizer)
        scaler.update()

        total_loss += loss.item()
        batches += 1
    scheduler.step()
    return total_loss / max(1, batches)

# Train
for _ in range(EPOCHS):
    _ = train_one_epoch()

end_time = time.time()

duration = end_time - start_time
accuracy = test_variable_delay_model(model)

print('duration: ', duration)
print('accuracy: ', accuracy)

assert end_time - start_time < 600, 'executing took longer than 10 mins'
assert accuracy > 0.99, 'accuracy is too low'
print('tests passed')

/tmp/ipykernel_13708/4028122569.py:38: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = torch.cuda.amp.GradScaler(enabled=(device.type == "cuda"))


/tmp/ipykernel_13708/4028122569.py:52: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(enabled=(device.type == "cuda")):


duration:  48.24670100212097
accuracy:  0.9994998124296611
tests passed
